In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [22]:
def get_city_name(link):
    return link.split('/')[-3]

In [4]:
df = pd.read_csv("./houses.csv", header=None, names=["Link"])

In [2]:
driver = webdriver.Firefox()

In [3]:
link = "https://www.pararius.com/apartment-for-rent/hilversum/bf750f4f/beresteinseweg"
driver.get(link)

In [4]:
html = driver.page_source
soup = BeautifulSoup(html)

In [17]:
def get_houses(link):
    try:
        houses_list = []
        driver.get(link)
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html)

        results = soup.find_all(class_="listing-search-item__link listing-search-item__link--depiction")

        for item in results:
            l = item.get('href')
            l = "https://www.pararius.com" + l
            houses_list.append(l)
        return houses_list
    except:
        return 0

In [18]:
houses_list = get_houses("https://www.pararius.com/apartments/utrecht/0-1300/radius-25/50m2")
houses_list

['https://www.pararius.com/apartment-for-rent/bodegraven/5ff26bd9/franciscushof',
 'https://www.pararius.com/apartment-for-rent/hilversum/e00821bc/koninginneweg',
 'https://www.pararius.com/apartment-for-rent/utrecht/cf00852e/winklerlaan',
 'https://www.pararius.com/apartment-for-rent/hilversum/bf750f4f/beresteinseweg',
 'https://www.pararius.com/apartment-for-rent/breukelen/7beb6620/brugstraat',
 'https://www.pararius.com/apartment-for-rent/zeist/9690ae70/utrechtseweg',
 'https://www.pararius.com/apartment-for-rent/zeist/fb404359/laan-van-rijnwijk',
 'https://www.pararius.com/house-for-rent/nieuwegein/efd85a7d/boomkruiper',
 'https://www.pararius.com/apartment-for-rent/weesp/59ea922b/reigersweide',
 'https://www.pararius.com/apartment-for-rent/utrecht/6834f491/willem-barentszstraat',
 'https://www.pararius.com/apartment-for-rent/hilversum/7d065d34/larenseweg',
 'https://www.pararius.com/apartment-for-rent/woudenberg/b2fb3f73/ridderhof',
 'https://www.pararius.com/apartment-for-rent/ze

In [19]:
df = pd.DataFrame(houses_list, columns=["link"])
df

,link
0,https://www.pararius.com/apartment-for-rent/bo...
1,https://www.pararius.com/apartment-for-rent/hi...
2,https://www.pararius.com/apartment-for-rent/ut...
3,https://www.pararius.com/apartment-for-rent/hi...
4,https://www.pararius.com/apartment-for-rent/br...
5,https://www.pararius.com/apartment-for-rent/ze...
6,https://www.pararius.com/apartment-for-rent/ze...
7,https://www.pararius.com/house-for-rent/nieuwe...
8,https://www.pararius.com/apartment-for-rent/we...
9,https://www.pararius.com/apartment-for-rent/ut...


In [30]:
def get_info(link):
    price, number, agent = -1,-1,None

    print('Starting', link)
    try:
        driver.get(link)
        # print('got page')
        time.sleep(5)
        html = driver.page_source
        soup = BeautifulSoup(html)
    except Exception as e:
        print('Driver error', e)
        return price, number, agent

    results = soup.find(class_="listing-detail-summary__price")
    if results:
        price = results.text
        price = price.split('\n')[-4]
        price = price.split('€')[-1]
        price = price.replace(",", "")
    else:
        print('Price error')

    results = soup.find(class_='agent-summary__link agent-summary__link--hidden agent-summary__link--call-agent')
    if results:
        number = results.text
        number = '+' + number.split('\n')[-2].split('+')[-1]
    else:
        print("Number error")

    results = soup.find(class_="agent-summary__title-link")
    if results:
        agent = results.text
    else:
        print('Agent error')

    values = soup.find_all(class_="listing-features__main-description")
    keys = soup.find_all(class_="listing-features__term")

    res = {}
    for key, value in zip(keys,values):
        res[key.text] = value.text

    res['price'] = price
    res['number'] = number
    res['agent'] = agent

    return res

In [23]:
df['city'] = df['link'].apply(get_city_name)
df

,link,city
0,https://www.pararius.com/apartment-for-rent/bo...,bodegraven
1,https://www.pararius.com/apartment-for-rent/hi...,hilversum
2,https://www.pararius.com/apartment-for-rent/ut...,utrecht
3,https://www.pararius.com/apartment-for-rent/hi...,hilversum
4,https://www.pararius.com/apartment-for-rent/br...,breukelen
5,https://www.pararius.com/apartment-for-rent/ze...,zeist
6,https://www.pararius.com/apartment-for-rent/ze...,zeist
7,https://www.pararius.com/house-for-rent/nieuwe...,nieuwegein
8,https://www.pararius.com/apartment-for-rent/we...,weesp
9,https://www.pararius.com/apartment-for-rent/ut...,utrecht


In [36]:
for i, row in df.iterrows():
    link = row['link']
    res = get_info(link)
    df.loc[i, res.keys()] = res.values()

Starting https://www.pararius.com/apartment-for-rent/bodegraven/5ff26bd9/franciscushof
Starting https://www.pararius.com/apartment-for-rent/hilversum/e00821bc/koninginneweg
Starting https://www.pararius.com/apartment-for-rent/utrecht/cf00852e/winklerlaan
Starting https://www.pararius.com/apartment-for-rent/hilversum/bf750f4f/beresteinseweg
Starting https://www.pararius.com/apartment-for-rent/breukelen/7beb6620/brugstraat
Starting https://www.pararius.com/apartment-for-rent/zeist/9690ae70/utrechtseweg
Starting https://www.pararius.com/apartment-for-rent/zeist/fb404359/laan-van-rijnwijk
Starting https://www.pararius.com/house-for-rent/nieuwegein/efd85a7d/boomkruiper
Starting https://www.pararius.com/apartment-for-rent/weesp/59ea922b/reigersweide
Starting https://www.pararius.com/apartment-for-rent/utrecht/6834f491/willem-barentszstraat
Starting https://www.pararius.com/apartment-for-rent/hilversum/7d065d34/larenseweg
Starting https://www.pararius.com/apartment-for-rent/woudenberg/b2fb3f7

In [38]:
df.columns

Index(['link', 'city', 'Rental price', 'Offered since', 'Status', 'Available',
       'Duration', 'Living area', 'Type of house', 'Type of construction',
       'Year of construction', 'Number of rooms', 'Number of bedrooms',
       'Number of bathrooms', 'Facilities', 'Balcony', 'Garden',
       'Energy rating', 'Present', 'price', 'number', 'agent',
       'Rental agreement', 'Deposit', 'Interior', 'Upkeep', 'Location',
       'Number of floors', 'Shed/Storeroom', 'Type of parking', 'Pets allowed',
       'Volume', 'Insulation', 'Heating', 'Hot water',
       'Central heating boiler', 'Service costs', 'Smoking allowed',
       'Target audience', 'Income requirement', 'Maximum number of residents',
       'Description'],
      dtype='object')

In [40]:
df

,link,city,Rental price,Offered since,Status,Available,Duration,Living area,Type of house,Type of construction,...,Insulation,Heating,Hot water,Central heating boiler,Service costs,Smoking allowed,Target audience,Income requirement,Maximum number of residents,Description
0,https://www.pararius.com/apartment-for-rent/bo...,bodegraven,"€1,115 per month",24-05-2023,For rent,Immediately,Minimum of 12 months,76 m²,Gallery flat,Existing building,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.pararius.com/apartment-for-rent/hi...,hilversum,"€1,100 per month",24-05-2023,For rent,From 15-06-2023,NaN,70 m²,"Downstairs apartment, Apartment",New development,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.pararius.com/apartment-for-rent/ut...,utrecht,"€1,080 per month",22-05-2023,For rent,From 01-07-2023,NaN,75 m²,Gallery flat,Existing building,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.pararius.com/apartment-for-rent/hi...,hilversum,"€1,000 per month",19-05-2023,For rent,From 01-06-2023,NaN,53 m²,"Gallery flat, Apartment",Existing building,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.pararius.com/apartment-for-rent/br...,breukelen,"€1,250 per month",17-05-2023,For rent,In consultation,NaN,72 m²,"Upstairs apartment, Apartment",Existing building,...,"Double glazing, Floor insulation, Roof insulation",Central heating boiler,Central heating boiler,"Remeha Combiketel (Gas, from 2018, Property)",NaN,NaN,NaN,NaN,NaN,NaN
5,https://www.pararius.com/apartment-for-rent/ze...,zeist,"€1,215 per month",17-05-2023,For rent,From 01-06-2023,NaN,70 m²,Downstairs apartment,Existing building,...,NaN,NaN,NaN,NaN,€185,No,Working,"€4,200",2,NaN
6,https://www.pararius.com/apartment-for-rent/ze...,zeist,"€1,035 per month",17-05-2023,For rent,From 01-07-2023,NaN,76 m²,"Downstairs apartment, Apartment",Existing building,...,Double glazing,Block heating,Gas boiler,NaN,€165,No,NaN,NaN,NaN,NaN
7,https://www.pararius.com/house-for-rent/nieuwe...,nieuwegein,"€1,045 per month",15-05-2023,For rent,From 01-06-2023,NaN,95 m²,"Mid-terrace house, Family home",Existing building,...,NaN,NaN,NaN,NaN,€15,No,Working,"€3,500",NaN,NaN
8,https://www.pararius.com/apartment-for-rent/we...,weesp,"€1,250 per month",15-05-2023,For rent,In consultation,NaN,52 m²,"Upstairs apartment, Apartment",Existing building,...,Double glazing,Central heating boiler,Central heating boiler,NaN,NaN,NaN,NaN,NaN,NaN,Aangebouwd steen
9,https://www.pararius.com/apartment-for-rent/ut...,utrecht,"€1,250 per month",12-05-2023,For rent,From 01-06-2023,NaN,57 m²,Upstairs apartment,Existing building,...,NaN,NaN,NaN,NaN,NaN,No,Working,"€4,200",1,NaN


In [41]:
df['Income requirement']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5     €4,200
6        NaN
7     €3,500
8        NaN
9     €4,200
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
20       NaN
21       NaN
22       NaN
23       NaN
24    €4,050
25    €3,075
26       NaN
27       NaN
28       NaN
29       NaN
Name: Income requirement, dtype: object

In [25]:
df.to_csv('houses-info-2.csv')